In [2]:
from minio import Minio
from minio.error import S3Error

client = Minio(
    "localhost:9000",  # Replace with your MinIO server address
    access_key="ml_testing",
    secret_key="ml_testing",
    secure=False  # Set to True if using HTTPS
)

# Create a bucket
bucket_name = "ml-bucket"
try:
    client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except S3Error as err:
    print(f"Error occurred: {err}")

Error occurred: S3 operation failed; code: BucketAlreadyOwnedByYou, message: Your previous request to create the named bucket succeeded and you already own it., resource: /ml-bucket, request_id: 182E12AAA3ED1AB0, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: ml-bucket


In [3]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as pltpoetry 


import random
import sys
from dvclive import Live
from sklearn.metrics import r2_score

In [4]:
df = pd.read_pickle('data/ev_pareto_on_gen.pkl')

# Split the data into features and target
X = df[["rotorAngle","rotorDiameter","Bias"]]
y = df[['LapTime','Durability','MotorPower','Torque']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:

with Live(save_dvc_exp=True) as live:
    
    for depth in range(2,5):

        live.log_metric("depth", depth)

        # Initialize the CatBoostRegressor
        model = CatBoostRegressor(
            iterations=2000, learning_rate=0.05, depth=depth, loss_function='MultiRMSE', verbose=200)

        # Fit the model
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_test)
        column_names = ['LapTime','Durability','MotorPower','Torque']
        predictions = model.predict(X_test)
        y_test_pred = pd.DataFrame(predictions,columns=column_names)

        live.log_metric("test/r2", r2_score(y_test['Torque'], y_test_pred['Torque']), plot=False)

        live.next_step()

0:	learn: 17351.1215652	total: 309us	remaining: 619ms
200:	learn: 595.8662059	total: 36.3ms	remaining: 325ms
400:	learn: 418.6189426	total: 72.2ms	remaining: 288ms
600:	learn: 332.5569872	total: 110ms	remaining: 256ms
800:	learn: 281.5669004	total: 147ms	remaining: 220ms
1000:	learn: 244.0919726	total: 184ms	remaining: 184ms
1200:	learn: 215.4948315	total: 222ms	remaining: 147ms
1400:	learn: 191.5262942	total: 258ms	remaining: 110ms
1600:	learn: 173.7236478	total: 294ms	remaining: 73.4ms
1800:	learn: 156.0415848	total: 331ms	remaining: 36.5ms
1999:	learn: 142.2351638	total: 367ms	remaining: 0us
0:	learn: 17353.4592358	total: 278us	remaining: 557ms
200:	learn: 462.4273572	total: 47ms	remaining: 420ms
400:	learn: 333.6030237	total: 91.8ms	remaining: 366ms
600:	learn: 259.1763461	total: 151ms	remaining: 351ms
800:	learn: 211.8377376	total: 220ms	remaining: 329ms
1000:	learn: 176.2015821	total: 267ms	remaining: 266ms
1200:	learn: 150.5525691	total: 318ms	remaining: 211ms
1400:	learn: 130.6